In [1]:
import os
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.data import Subset
import torch.optim as optim
import torch.nn.functional as F

In [2]:
# === CONFIG ===
INPUT_DIR = "/home/HardDisk/Satang/thesis_proj/New_45/15/split_tws/X_csv_split_31"
CHUNK_SIZE = 31
NUM_FEATURES = 8
NUM_EPOCHS = 50
BATCH_SIZE = 32
K_FOLDS = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# === HELPERS ===
def collate_fn(batch):
    streams, labels = zip(*batch)
    streams = list(zip(*streams))
    streams = [torch.stack(s) for s in streams]
    labels = torch.tensor(labels)
    return streams, labels

def compute_class_weights(labels, device):
    counter = Counter(labels)
    total = sum(counter.values())
    weights = [np.log(total / (counter[i] + 1)) for i in range(len(counter))]
    return torch.tensor(weights, dtype=torch.float).to(device)

def print_model_info(model):
    print("Model Architecture:")
    print(model)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total parameters: {total_params:,}")

def plot_loss_curve(train_losses, val_losses):
    epochs = range(len(train_losses))
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, train_losses, label='Train Loss', color='blue')
    plt.plot(epochs, val_losses, label='Val Loss', color='orange')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training vs Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_accuracy_curve(train_acc, val_acc):
    epochs = range(len(train_acc))
    plt.figure(figsize=(10, 5))
    plt.plot(epochs, train_acc, label='Train Accuracy', color='green')
    plt.plot(epochs, val_acc, label='Val Accuracy', color='red')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training vs Validation Accuracy')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [4]:
# === DATASET CLASS ===
class MultiStreamDataset(Dataset):
    def __init__(self, data, labels, label_encoder, augment=False):
        self.data = data
        self.labels = label_encoder.transform(labels)
        self.augment = augment

    def __len__(self):
        return len(self.data)

    def augment_stream(self, stream):
        jitter = np.random.normal(0, 0.01, stream.shape)
        scale = np.random.normal(1.0, 0.05, stream.shape)
        return stream * scale + jitter

    def __getitem__(self, idx):
        sample = self.data[idx]  # shape: (T, 8)
        
        if self.augment:
            # Apply augmentation to each feature independently
            jitter = np.random.normal(0, 0.01, sample.shape)
            scale = np.random.normal(1.0, 0.05, sample.shape)
            sample = sample * scale + jitter

        sample_tensor = torch.tensor(sample, dtype=torch.float32)  # shape: (T, 8)
        label_tensor = torch.tensor(self.labels[idx], dtype=torch.long)
    
        return sample_tensor, label_tensor


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PatchMLP(nn.Module):
    def __init__(self, dim, hidden_dim):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, dim)
        )

    def forward(self, x):
        return self.mlp(x)

class ChannelMLP(nn.Module):
    def __init__(self, num_patches, hidden_dim):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(num_patches, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, num_patches)
        )

    def forward(self, x):
        return self.mlp(x)

class MLPMixerRansomwareClassifier(nn.Module):
    def __init__(self, seq_len=32, feature_dim=8, num_classes=12, hidden_dim=128):
        super().__init__()
        self.patch_mlp = PatchMLP(feature_dim, hidden_dim)
        self.channel_mlp = ChannelMLP(seq_len, hidden_dim)
        self.global_pool = nn.AdaptiveAvgPool1d(1)

        self.projection = nn.Linear(feature_dim, 128)  # for KD
        self.classifier = nn.Sequential(
            nn.LayerNorm(feature_dim),
            nn.Flatten(),
            nn.Linear(feature_dim, num_classes)
        )

    def forward(self, x, return_features=False):  # x: (B, T, 8)
        x = self.patch_mlp(x)          # (B, T, 8)
        x = x.permute(0, 2, 1)         # (B, 8, T)
        x = self.channel_mlp(x)        # (B, 8, T)

        x = self.global_pool(x)        # (B, 8, 1)
        feat = x.squeeze(-1)           # (B, 8)

        feat_proj = self.projection(feat)      # (B, 128)
        logits = self.classifier(feat)         # (B, num_classes)

        if return_features:
            return logits, feat_proj
        else:
            return logits


In [6]:

# # === COLLATE FUNCTION ===
# def collate_fn(batch):
#     streams, labels = zip(*batch)
#     streams = list(zip(*streams))
#     streams = [torch.stack(s) for s in streams]
#     labels = torch.tensor(labels)
#     return streams, labels

# === LOAD SPLIT FUNCTION ===
def load_split_from_folder(split_dir, expected_shape):
    X, y = [], []
    for class_name in sorted(os.listdir(split_dir)):
        class_path = os.path.join(split_dir, class_name)
        for fname in sorted(os.listdir(class_path)):
            if fname.endswith(".csv"):
                fpath = os.path.join(class_path, fname)
                chunk = pd.read_csv(fpath, header=None).values
                if chunk.shape == expected_shape:
                    X.append(chunk)
                    y.append(class_name)
    return np.array(X), np.array(y)

In [7]:
# === APPLY SMOTE ===
def apply_smote_on_training(X_chunks, y_labels, chunk_size, num_features):
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_labels)

    smote_encoder = LabelEncoder()
    y_encoded_for_smote = smote_encoder.fit_transform(y_labels)

    X_flat = X_chunks.reshape(X_chunks.shape[0], -1)
    X_resampled, y_resampled = SMOTE().fit_resample(X_flat, y_encoded_for_smote)

    X_res = X_resampled.reshape(-1, chunk_size, num_features)
    y_res_str = smote_encoder.inverse_transform(y_resampled)

    return X_res, y_res_str, label_encoder

# # === LOAD DATASETS ===
# expected_shape = (CHUNK_SIZE, NUM_FEATURES)

# X_train_raw, y_train_raw = load_split_from_folder(os.path.join(INPUT_DIR, "train"), expected_shape)
# X_val_raw,   y_val_raw   = load_split_from_folder(os.path.join(INPUT_DIR, "val"), expected_shape)

# # === SMOTE ONLY ON TRAIN ===
# X_train_balanced, y_train_str, label_encoder = apply_smote_on_training(
#     X_train_raw, y_train_raw, CHUNK_SIZE, NUM_FEATURES
# )

# # === CREATE DATASETS ===
# train_dataset = MultiStreamDataset(X_train_balanced, y_train_str, label_encoder, augment=True)
# val_dataset   = MultiStreamDataset(X_val_raw, y_val_raw, label_encoder, augment=False)

# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
# val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [8]:
# === CLASS WEIGHTING ===
def compute_class_weights(labels, device):
    from collections import Counter
    total = len(labels)
    counts = Counter(labels)
    weights = [np.log(total / (counts[i] + 1)) for i in range(len(counts))]
    return torch.tensor(weights, dtype=torch.float).to(device)

# class_weights_tensor = compute_class_weights(label_encoder.transform(y_train_str), device="cpu")

In [9]:
def train_model(model, train_loader, val_loader, device, epochs=50, lr=0.001,
                  class_weights=None, optimizer=None, scheduler=None,
                  best_model_path="mlpmixer_teacher.pth"):

    print("🔍 Class Weights Tensor:")
    print(class_weights)

    criterion_ce = nn.CrossEntropyLoss(weight=class_weights)

    if optimizer is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    model.to(device)

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    best_val_acc = 0

    for epoch in range(epochs):
        model.train()
        total_loss, correct = 0.0, 0

        if scheduler:
            scheduler.step()

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            logits = model(inputs)

            loss = criterion_ce(logits, labels)

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
            optimizer.step()

            total_loss += loss.item()
            correct += (logits.argmax(1) == labels).sum().item()

        train_acc = correct / len(train_loader.dataset)
        train_losses.append(total_loss)
        train_accuracies.append(train_acc)

        # === Validation ===
        model.eval()
        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                logits = model(inputs)

                loss = criterion_ce(logits, labels)
                val_loss += loss.item()
                val_correct += (logits.argmax(1) == labels).sum().item()

        val_acc = val_correct / len(val_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(f"Epoch {epoch+1}/{epochs} - "
              f"Train Loss: {total_loss:.4f} - Val Loss: {val_loss:.4f} - "
              f"Train Acc: {train_acc:.4f} - Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_model_path)
            print(f"💾 Best model saved to: {best_model_path}")

    model.load_state_dict(torch.load(best_model_path))
    return train_accuracies, val_accuracies, train_losses, val_losses



In [10]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from imblearn.over_sampling import SMOTE

# === 🧩 Config ===
NUM_EPOCHS = 50
BEST_LR = 0.01
NUM_FEATURES = 8  # Assuming you use 8D input
BATCH_SIZE = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

base_dir = "/home/HardDisk/Satang/thesis_proj"
save_root = "/home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/teacher"
os.makedirs(save_root, exist_ok=True)

detection_times = [30, 45, 60]
window_sizes = [10, 15, 20]

for T_d in detection_times:
    for T_w in window_sizes:
        print(f"\n🚀 Running for Td={T_d}, Tw={T_w}")
        model_name = os.path.join(save_root, f"mlpmixer_Td{T_d}_Tw{T_w}.pth")
        T_len = T_d - T_w + 1
        folder_name = f"X_csv_split_{T_len}"

        input_dir = os.path.join(base_dir, f"New_{T_d}", f"{T_w}", "split_tws", folder_name)
        train_path = os.path.join(input_dir, "train")
        val_path   = os.path.join(input_dir, "val")

        # === 1. Load Data ===
        expected_shape = (T_len, NUM_FEATURES)
        X_train_raw, y_train_raw = load_split_from_folder(train_path, expected_shape)
        X_val_raw, y_val_raw     = load_split_from_folder(val_path, expected_shape)

        # === 2. SMOTE + Encode ===
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train_raw)
        X_train_flat = X_train_raw.reshape(X_train_raw.shape[0], -1)
        X_resampled, y_resampled = SMOTE().fit_resample(X_train_flat, y_train_encoded)
        X_train_bal = X_resampled.reshape(-1, expected_shape[0], NUM_FEATURES)
        y_train_str = label_encoder.inverse_transform(y_resampled)

        # === 3. Datasets + Loaders ===
        train_dataset = MultiStreamDataset(X_train_bal, y_train_str, label_encoder, augment=True)
        val_dataset   = MultiStreamDataset(X_val_raw, y_val_raw, label_encoder, augment=False)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # === 4. MLP-Mixer Model ===
        model = MLPMixerRansomwareClassifier(
            seq_len=expected_shape[0],
            feature_dim=NUM_FEATURES,
            num_classes=len(label_encoder.classes_),
            hidden_dim=128
        ).to(device)
        optimizer = optim.AdamW(model.parameters(), lr=BEST_LR, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)
        class_weights_tensor = compute_class_weights(label_encoder.transform(y_train_str), device)

        # === 5. Train MLP-Mixer ===
        train_accs, val_accs, train_losses, val_losses = train_model(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            device=device,
            epochs=NUM_EPOCHS,
            class_weights=class_weights_tensor,
            lr=BEST_LR,
            optimizer=optimizer,
            scheduler=scheduler,
            best_model_path=model_name
        )

        print(f"✅ Finished Td={T_d}, Tw={T_w} — saved to {model_name}")



🚀 Running for Td=30, Tw=10
🔍 Class Weights Tensor:
tensor([2.4843, 2.4843, 2.4843, 2.4843, 2.4843, 2.4843, 2.4843, 2.4843, 2.4843,
        2.4843, 2.4843, 2.4843], device='cuda:0')


/home/ea301b/anaconda3/envs/dl_satang/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 1/50 - Train Loss: 336.0935 - Val Loss: 35.1825 - Train Acc: 0.5693 - Val Acc: 0.5486
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/teacher/mlpmixer_Td30_Tw10.pth
Epoch 2/50 - Train Loss: 221.1273 - Val Loss: 28.9831 - Train Acc: 0.7053 - Val Acc: 0.6187
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/teacher/mlpmixer_Td30_Tw10.pth
Epoch 3/50 - Train Loss: 190.3074 - Val Loss: 27.8167 - Train Acc: 0.7472 - Val Acc: 0.6508
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/teacher/mlpmixer_Td30_Tw10.pth
Epoch 4/50 - Train Loss: 165.7556 - Val Loss: 25.9019 - Train Acc: 0.7779 - Val Acc: 0.6628
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/teacher/mlpmixer_Td30_Tw10.pth
Epoch 5/50 - Train Loss: 146.8350 - Val Loss: 22.4268 - Train Acc: 0.8091 - Val Acc: 0.7425
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learni

/tmp/ipykernel_2429355/2461672243.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


🔍 Class Weights Tensor:
tensor([2.4844, 2.4844, 2.4844, 2.4844, 2.4844, 2.4844, 2.4844, 2.4844, 2.4844,
        2.4844, 2.4844, 2.4844], device='cuda:0')


...

/tmp/ipykernel_2429355/2461672243.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


🔍 Class Weights Tensor:
tensor([2.4835, 2.4835, 2.4835, 2.4835, 2.4835, 2.4835, 2.4835, 2.4835, 2.4835,
        2.4835, 2.4835, 2.4835], device='cuda:0')


/home/ea301b/anaconda3/envs/dl_satang/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Epoch 1/50 - Train Loss: 215.0630 - Val Loss: 16.3670 - Train Acc: 0.4426 - Val Acc: 0.5316
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/teacher/mlpmixer_Td60_Tw20.pth
Epoch 2/50 - Train Loss: 92.5499 - Val Loss: 11.1087 - Train Acc: 0.7667 - Val Acc: 0.6698
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/teacher/mlpmixer_Td60_Tw20.pth
Epoch 3/50 - Train Loss: 66.9137 - Val Loss: 8.6636 - Train Acc: 0.8243 - Val Acc: 0.7518
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/teacher/mlpmixer_Td60_Tw20.pth
Epoch 4/50 - Train Loss: 55.2634 - Val Loss: 7.6946 - Train Acc: 0.8544 - Val Acc: 0.7904
💾 Best model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/teacher/mlpmixer_Td60_Tw20.pth
Epoch 5/50 - Train Loss: 45.9841 - Val Loss: 8.2227 - Train Acc: 0.8823 - Val Acc: 0.7506
Epoch 6/50 - Train Loss: 43.4650 - Val Loss: 7.7386 - Train Acc: 0.8857 - V

/tmp/ipykernel_2429355/2461672243.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


In [11]:
import torch
import torch.nn as nn

class StudentCNN(nn.Module):
    def __init__(self, input_length, num_classes=12):
        super().__init__()
        self.streams = nn.ModuleList([
            nn.Sequential(
                nn.Conv1d(1, 4, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.Conv1d(4, 8, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.AdaptiveAvgPool1d(1),
                nn.Flatten()
            ) for _ in range(8)
        ])

        self.fc = nn.Sequential(
            nn.Linear(8 * 8, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )

        self.proj = nn.Linear(8 * 8, 128)

    def forward(self, x, return_features=False):
        B, T, C = x.shape
        assert C == 8, f"Expected 8 feature streams, got {C}"

        # Split into 8 streams (B, 1, T) → process → (B, 8)
        streams = [x[:, :, i].unsqueeze(1) for i in range(C)]
        features = [self.streams[i](streams[i]) for i in range(8)]

        x = torch.cat(features, dim=1)  # (B, 64)



        if return_features:
            feat_proj = self.proj(x)      # (B, 128)
            logits = self.fc(x)           # (B, num_classes)
            return logits, feat_proj
        else:
            return self.fc(x)


In [12]:
import torch.nn.functional as F

def distillation_loss(student_logits, teacher_logits, student_feat, teacher_feat, true_labels, T=3.0, alpha=0.5, beta=0.3, gamma=0.2):
    """
    alpha: weight for hard loss (CE)
    beta: weight for soft loss (KL)
    gamma: weight for feature-based distillation (MSE)
    T: temperature for soft distillation
    """
    # Hard loss
    ce_loss = F.cross_entropy(student_logits, true_labels)

    # Soft loss (logits)
    soft_teacher = F.softmax(teacher_logits / T, dim=1)
    soft_student = F.log_softmax(student_logits / T, dim=1)
    kl_loss = F.kl_div(soft_student, soft_teacher, reduction='batchmean') * (T * T)

    # Feature loss (projection-matched MSE)
    feat_loss = F.mse_loss(student_feat, teacher_feat)

    return alpha * ce_loss + beta * kl_loss + gamma * feat_loss


In [13]:
def train_distilled(student, teacher, train_loader, val_loader, device,
                    epochs=50, lr=0.0005, class_weights=None,
                    T=3.0, alpha=0.5, beta=0.3, gamma=0.2,
                    save_path="best_student.pth"):

    teacher.eval()
    student.to(device)
    teacher.to(device)

    optimizer = torch.optim.Adam(student.parameters(), lr=lr)
    ce_criterion = nn.CrossEntropyLoss(weight=class_weights)

    best_val_acc = 0
    train_losses, train_accuracies, val_losses, val_accuracies = [], [], [], []

    for epoch in range(epochs):
        student.train()
        total_loss, correct = 0.0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # === Forward passes (both must return logits, features)
            student_logits, student_feat = student(inputs, return_features=True)
            with torch.no_grad():
                teacher_logits, teacher_feat = teacher(inputs, return_features=True)

            # === KD loss
            loss = distillation_loss(
                student_logits, teacher_logits,
                student_feat, teacher_feat,
                labels,
                T=T, alpha=alpha, beta=beta, gamma=gamma
            )

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            correct += (student_logits.argmax(1) == labels).sum().item()

        train_acc = correct / len(train_loader.dataset)
        train_losses.append(total_loss)
        train_accuracies.append(train_acc)

        # === Validation (using standard CE loss)
        student.eval()
        val_loss, val_correct = 0.0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                logits = student(inputs)  # ❗ Note: assuming return_features=False in eval
                loss = ce_criterion(logits, labels)
                val_loss += loss.item()
                val_correct += (logits.argmax(1) == labels).sum().item()

        val_acc = val_correct / len(val_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(f"[Distill] Epoch {epoch+1:02d}/{epochs} - "
              f"Loss: {total_loss:.4f} - Train Acc: {train_acc:.4f} - Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(student.state_dict(), save_path)
            print(f"💾 Best student model saved to: {save_path}")

    student.load_state_dict(torch.load(save_path))
    return train_accuracies, val_accuracies, train_losses, val_losses


In [24]:
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
import os
import torch
from torch.utils.data import DataLoader

# === Configs ===
NUM_FEATURES = 8
BATCH_SIZE = 64
NUM_EPOCHS = 50
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

detection_times = [30, 45, 60]
window_sizes = [10, 15, 20]

base_dir = "/home/HardDisk/Satang/thesis_proj"
teacher_dir = os.path.join(base_dir, "Deep_Learning", "cross_archi","mlp", "teacher")  # ← not AE
student_save_dir = os.path.join(base_dir, "Deep_Learning", "cross_archi","mlp", "student")
os.makedirs(student_save_dir, exist_ok=True)

for T_d in detection_times:
    for T_w in window_sizes:
        T_len = T_d - T_w + 1
        expected_shape = (T_len, NUM_FEATURES)

        print(f"\n🚀 Distilling MLP-Mixer → StudentCNN for Td={T_d}, Tw={T_w} (T_len={T_len})")

        # === Paths
        folder_name = f"X_csv_split_{T_len}"
        input_dir = os.path.join(base_dir, f"New_{T_d}", f"{T_w}", "split_tws", folder_name)
        train_path = os.path.join(input_dir, "train")
        val_path   = os.path.join(input_dir, "val")
        teacher_path = os.path.join(teacher_dir, f"mlpmixer_Td{T_d}_Tw{T_w}.pth")
        student_path = os.path.join(student_save_dir, f"student_from_mlp_Td{T_d}_Tw{T_w}.pth")

        # === 1. Load Data
        X_train_raw, y_train_raw = load_split_from_folder(train_path, expected_shape)
        X_val_raw, y_val_raw     = load_split_from_folder(val_path, expected_shape)

        # === 2. Encode & Balance
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train_raw)
        X_train_flat = X_train_raw.reshape(X_train_raw.shape[0], -1)
        X_resampled, y_resampled = SMOTE().fit_resample(X_train_flat, y_train_encoded)
        X_train_bal = X_resampled.reshape(-1, T_len, NUM_FEATURES)
        y_train_str = label_encoder.inverse_transform(y_resampled)

        # === 3. Dataset & Dataloader
        train_dataset = MultiStreamDataset(X_train_bal, y_train_str, label_encoder, augment=True)
        val_dataset   = MultiStreamDataset(X_val_raw, y_val_raw, label_encoder, augment=False)
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        class_weights_tensor = compute_class_weights(label_encoder.transform(y_train_str), device)

        # === 4. Load MLP-Mixer Teacher
        teacher = MLPMixerRansomwareClassifier(
            seq_len=T_len,
            feature_dim=NUM_FEATURES,
            num_classes=len(label_encoder.classes_),
            hidden_dim=128,

        )
        teacher.load_state_dict(torch.load(teacher_path, map_location=device))
        teacher.to(device)
        teacher.eval()
        for p in teacher.parameters():
            p.requires_grad = False

        # === 5. Init StudentCNN
        student = StudentCNN(
            input_length=T_len,
            num_classes=len(label_encoder.classes_)
        ).to(device)

        # === 6. Train with Knowledge Distillation
        train_accs, val_accs, train_losses, val_losses = train_distilled(
            student=student,
            teacher=teacher,
            train_loader=train_loader,
            val_loader=val_loader,
            device=device,
            epochs=NUM_EPOCHS,
            lr=0.01,
            class_weights=class_weights_tensor,
            T=3,
            alpha=0.6,
            beta=0.3,
            gamma=0.3,
            save_path=student_path
        )

        # === 7. Final Evaluation
        student.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)  # (logits only)
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        print("\n📊 Final Classification Report (StudentCNN):")
        print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))
        print(f"✅ Student model saved: {student_path}")



🚀 Distilling MLP-Mixer → StudentCNN for Td=30, Tw=10 (T_len=21)


/tmp/ipykernel_2429355/4218594340.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  teacher.load_state_dict(torch.load(teacher_path, map_location=device))


[Distill] Epoch 01/50 - Loss: 103172.3219 - Train Acc: 0.4912 - Val Acc: 0.5541
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/student/student_from_mlp_Td30_Tw10.pth
[Distill] Epoch 02/50 - Loss: 51221.1766 - Train Acc: 0.6715 - Val Acc: 0.5902
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/student/student_from_mlp_Td30_Tw10.pth
[Distill] Epoch 03/50 - Loss: 43765.6398 - Train Acc: 0.6981 - Val Acc: 0.6358
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/student/student_from_mlp_Td30_Tw10.pth
[Distill] Epoch 04/50 - Loss: 41058.7328 - Train Acc: 0.7351 - Val Acc: 0.6478
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/student/student_from_mlp_Td30_Tw10.pth
[Distill] Epoch 05/50 - Loss: 39624.7720 - Train Acc: 0.7501 - Val Acc: 0.6498
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Dee

/tmp/ipykernel_2429355/3282436521.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))


              precision    recall  f1-score   support

    AESCrypt       1.00      1.00      1.00        85
      Cerber       0.73      0.77      0.75       189
    Darkside       0.96      0.75      0.85       323
       Excel       1.00      1.00      1.00       147
     Firefox       1.00      0.99      1.00       204
   GandCrab4       0.71      0.78      0.74       319
        Ryuk       0.76      0.77      0.76       196
     SDelete       1.00      1.00      1.00        79
  Sodinokibi       0.79      0.73      0.76       205
  TeslaCrypt       0.65      1.00      0.79        85
    WannaCry       1.00      1.00      1.00        79
         Zip       1.00      1.00      1.00        85

    accuracy                           0.85      1996
   macro avg       0.88      0.90      0.89      1996
weighted avg       0.86      0.85      0.85      1996

✅ Student model saved: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/student/student_from_mlp_Td30_Tw10.pth

🚀 Dist

...

/tmp/ipykernel_2429355/3282436521.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))
/tmp/ipykernel_2429355/4218594340.py:65

[Distill] Epoch 01/50 - Loss: 8624.7069 - Train Acc: 0.6049 - Val Acc: 0.6405
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/student/student_from_mlp_Td60_Tw15.pth
[Distill] Epoch 02/50 - Loss: 3568.0302 - Train Acc: 0.7772 - Val Acc: 0.6949
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/student/student_from_mlp_Td60_Tw15.pth
[Distill] Epoch 03/50 - Loss: 3184.7275 - Train Acc: 0.8168 - Val Acc: 0.7468
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/student/student_from_mlp_Td60_Tw15.pth
[Distill] Epoch 04/50 - Loss: 2902.1499 - Train Acc: 0.8437 - Val Acc: 0.7987
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/student/student_from_mlp_Td60_Tw15.pth
[Distill] Epoch 05/50 - Loss: 2702.2410 - Train Acc: 0.8486 - Val Acc: 0.7519
[Distill] Epoch 06/50 - Loss: 2584.3240 - Train Acc: 0.8642 - Val Acc: 0.7

/tmp/ipykernel_2429355/3282436521.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))
/tmp/ipykernel_2429355/4218594340.py:65

[Distill] Epoch 01/50 - Loss: 16039.2905 - Train Acc: 0.4151 - Val Acc: 0.3326
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/student/student_from_mlp_Td60_Tw20.pth
[Distill] Epoch 02/50 - Loss: 5970.4138 - Train Acc: 0.6829 - Val Acc: 0.6417
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/student/student_from_mlp_Td60_Tw20.pth
[Distill] Epoch 03/50 - Loss: 4851.9650 - Train Acc: 0.7727 - Val Acc: 0.7564
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/student/student_from_mlp_Td60_Tw20.pth
[Distill] Epoch 04/50 - Loss: 4442.7708 - Train Acc: 0.7982 - Val Acc: 0.7014
[Distill] Epoch 05/50 - Loss: 4041.2446 - Train Acc: 0.8319 - Val Acc: 0.7670
💾 Best student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/student/student_from_mlp_Td60_Tw20.pth
[Distill] Epoch 06/50 - Loss: 3780.3928 - Train Acc: 0.8505 - Val Acc: 0.

/tmp/ipykernel_2429355/3282436521.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))


In [25]:
import torch
import torch.nn as nn
import torch.optim as optim

def train_student_baseline(student, train_loader, val_loader, device,
                           epochs=50, lr=0.0005, class_weights=None,
                           save_path="best_student_baseline.pth"):

    student.to(device)
    optimizer = optim.Adam(student.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(weight=class_weights)

    best_val_acc = 0.0
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    for epoch in range(epochs):
        student.train()
        running_loss, correct = 0.0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            logits = student(inputs)  # 🔹 logits only for baseline
            loss = criterion(logits, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)  # weighted sum
            correct += (logits.argmax(1) == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = correct / len(train_loader.dataset)
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)

        # === Validation ===
        student.eval()
        val_running_loss, val_correct = 0.0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                logits = student(inputs)
                loss = criterion(logits, labels)

                val_running_loss += loss.item() * inputs.size(0)
                val_correct += (logits.argmax(1) == labels).sum().item()

        val_loss = val_running_loss / len(val_loader.dataset)
        val_acc = val_correct / len(val_loader.dataset)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        print(f"[Baseline] Epoch {epoch+1:02d}/{epochs} - "
              f"Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f} - "
              f"Train Acc: {train_acc:.4f} - Val Acc: {val_acc:.4f}")

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(student.state_dict(), save_path)
            print(f"💾 Best standalone student model saved to: {save_path}")

    student.load_state_dict(torch.load(save_path))
    return train_accuracies, val_accuracies, train_losses, val_losses


In [26]:
import os
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from torch.utils.data import DataLoader

baseline_save_dir = "/home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/baseline_student"
os.makedirs(baseline_save_dir, exist_ok=True)

for T_d in detection_times:
    for T_w in window_sizes:
        T_len = T_d - T_w + 1
        expected_shape = (T_len, NUM_FEATURES)

        print(f"\n🚀 Training Standalone Student for Td={T_d}, Tw={T_w} (T_len={T_len})")

        # === Paths
        folder_name = f"X_csv_split_{T_len}"
        input_dir = os.path.join(base_dir, f"New_{T_d}", f"{T_w}", "split_tws", folder_name)
        train_path = os.path.join(input_dir, "train")
        val_path   = os.path.join(input_dir, "val")
        student_model_path = os.path.join(baseline_save_dir, f"student_baseline_Td{T_d}_Tw{T_w}.pth")

        # === 1. Load Raw Data
        X_train_raw, y_train_raw = load_split_from_folder(train_path, expected_shape)
        X_val_raw, y_val_raw     = load_split_from_folder(val_path, expected_shape)

        # === 2. Encode Labels & Apply SMOTE
        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train_raw)
        X_train_flat = X_train_raw.reshape(X_train_raw.shape[0], -1)

        X_resampled, y_resampled = SMOTE().fit_resample(X_train_flat, y_train_encoded)
        X_train_bal = X_resampled.reshape(-1, T_len, NUM_FEATURES)
        y_train_str = label_encoder.inverse_transform(y_resampled)

        # === 3. Dataset & DataLoader
        train_dataset = MultiStreamDataset(X_train_bal, y_train_str, label_encoder, augment=True)
        val_dataset   = MultiStreamDataset(X_val_raw, y_val_raw, label_encoder, augment=False)

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader   = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # === 4. Class Weights
        class_weights_tensor = compute_class_weights(label_encoder.transform(y_train_str), device)

        # === 5. Initialize StudentCNN
        student_baseline = StudentCNN(
            input_length=T_len,
            num_classes=len(label_encoder.classes_)
        ).to(device)

        # === 6. Train Student Model (No KD)
        train_accs_b, val_accs_b, train_losses_b, val_losses_b = train_student_baseline(
            student=student_baseline,
            train_loader=train_loader,
            val_loader=val_loader,
            device=device,
            epochs=NUM_EPOCHS,
            lr=0.0001,
            class_weights=class_weights_tensor,
            save_path=student_model_path
        )

        print(f"✅ Saved standalone student model: {student_model_path}")



🚀 Training Standalone Student for Td=30, Tw=10 (T_len=21)


[Baseline] Epoch 01/50 - Train Loss: 2.2955 - Val Loss: 1.9555 - Train Acc: 0.1961 - Val Acc: 0.3141
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/baseline_student/student_baseline_Td30_Tw10.pth
[Baseline] Epoch 02/50 - Train Loss: 1.5758 - Val Loss: 1.5551 - Train Acc: 0.4764 - Val Acc: 0.4765
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/baseline_student/student_baseline_Td30_Tw10.pth
[Baseline] Epoch 03/50 - Train Loss: 1.2834 - Val Loss: 1.3521 - Train Acc: 0.5754 - Val Acc: 0.5476
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/baseline_student/student_baseline_Td30_Tw10.pth
[Baseline] Epoch 04/50 - Train Loss: 1.0933 - Val Loss: 1.2085 - Train Acc: 0.6357 - Val Acc: 0.5691
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/baseline_student/student_bas

/tmp/ipykernel_2429355/3582298853.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))


[Baseline] Epoch 01/50 - Train Loss: 2.4214 - Val Loss: 2.2727 - Train Acc: 0.1514 - Val Acc: 0.2965
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/baseline_student/student_baseline_Td30_Tw15.pth
[Baseline] Epoch 02/50 - Train Loss: 1.7378 - Val Loss: 1.6632 - Train Acc: 0.4360 - Val Acc: 0.4590
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/baseline_student/student_baseline_Td30_Tw15.pth
[Baseline] Epoch 03/50 - Train Loss: 1.3216 - Val Loss: 1.4112 - Train Acc: 0.5892 - Val Acc: 0.5399
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/baseline_student/student_baseline_Td30_Tw15.pth
[Baseline] Epoch 04/50 - Train Loss: 1.1076 - Val Loss: 1.2449 - Train Acc: 0.6565 - Val Acc: 0.5866
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/baseline_student/student_bas

/tmp/ipykernel_2429355/3582298853.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))


...

/tmp/ipykernel_2429355/3582298853.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))


[Baseline] Epoch 01/50 - Train Loss: 2.5073 - Val Loss: 2.4620 - Train Acc: 0.0994 - Val Acc: 0.1443
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/baseline_student/student_baseline_Td60_Tw15.pth
[Baseline] Epoch 02/50 - Train Loss: 2.3875 - Val Loss: 2.3619 - Train Acc: 0.2119 - Val Acc: 0.2924
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/baseline_student/student_baseline_Td60_Tw15.pth
[Baseline] Epoch 03/50 - Train Loss: 2.0764 - Val Loss: 1.9494 - Train Acc: 0.3823 - Val Acc: 0.4000
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/baseline_student/student_baseline_Td60_Tw15.pth
[Baseline] Epoch 04/50 - Train Loss: 1.5613 - Val Loss: 1.6389 - Train Acc: 0.5268 - Val Acc: 0.5000
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/baseline_student/student_bas

/tmp/ipykernel_2429355/3582298853.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))


[Baseline] Epoch 01/50 - Train Loss: 2.4633 - Val Loss: 2.4337 - Train Acc: 0.1136 - Val Acc: 0.1370
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/baseline_student/student_baseline_Td60_Tw20.pth
[Baseline] Epoch 02/50 - Train Loss: 2.2445 - Val Loss: 2.0877 - Train Acc: 0.2811 - Val Acc: 0.3407
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/baseline_student/student_baseline_Td60_Tw20.pth
[Baseline] Epoch 03/50 - Train Loss: 1.6023 - Val Loss: 1.6121 - Train Acc: 0.5627 - Val Acc: 0.5375
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/baseline_student/student_baseline_Td60_Tw20.pth
[Baseline] Epoch 04/50 - Train Loss: 1.2694 - Val Loss: 1.3703 - Train Acc: 0.6687 - Val Acc: 0.6077
💾 Best standalone student model saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/baseline_student/student_bas

/tmp/ipykernel_2429355/3582298853.py:66: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  student.load_state_dict(torch.load(save_path))


In [27]:
import os
import torch
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

# === Model Definition ===
class PatchMLP(nn.Module):
    def __init__(self, dim, hidden_dim):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, dim)
        )

    def forward(self, x):
        return self.mlp(x)

class ChannelMLP(nn.Module):
    def __init__(self, num_patches, hidden_dim):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(num_patches, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, num_patches)
        )

    def forward(self, x):
        return self.mlp(x)

class MLPMixerRansomwareClassifier(nn.Module):
    def __init__(self, seq_len=32, feature_dim=8, num_classes=12, hidden_dim=128):
        super().__init__()
        self.patch_mlp = PatchMLP(feature_dim, hidden_dim)
        self.channel_mlp = ChannelMLP(seq_len, hidden_dim)
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.projection = nn.Linear(feature_dim, 128)
        self.classifier = nn.Sequential(
            nn.LayerNorm(feature_dim),
            nn.Flatten(),
            nn.Linear(feature_dim, num_classes)
        )

    def forward(self, x, return_features=False):
        x = self.patch_mlp(x)
        x = x.permute(0, 2, 1)
        x = self.channel_mlp(x)
        x = self.global_pool(x)
        feat = x.squeeze(-1)
        feat_proj = self.projection(feat)
        logits = self.classifier(feat)
        return (logits, feat_proj) if return_features else logits

# === CONFIG ===
detection_times = [30, 45, 60]
window_sizes = [10, 15, 20]
NUM_FEATURES = 8
BATCH_SIZE = 32
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

base_dir = "/home/HardDisk/Satang/thesis_proj/"
base_teacher_dir = os.path.join(base_dir, "Deep_Learning", "cross_archi", "mlp", "teacher")
student_kd_dir = os.path.join(base_dir, "Deep_Learning","cross_archi", "mlp", "student")
student_baseline_dir = os.path.join(base_dir, "Deep_Learning", "cross_archi","mlp", "baseline_student")
csv_output_path = os.path.join(base_dir, "Deep_Learning", "cross_archi","mlp", "results", "model_eval_summary.csv")
os.makedirs(os.path.dirname(csv_output_path), exist_ok=True)

results = []

# === Evaluation Function ===
def evaluate_and_log(model, model_path, test_loader, label_encoder, T_d, T_w, model_type, results_list):
    model.load_state_dict(torch.load(model_path, map_location=DEVICE))
    model.to(DEVICE)
    model.eval()

    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            logits, _ = model(inputs, return_features=True)
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    report = classification_report(all_labels, all_preds, target_names=label_encoder.classes_, output_dict=True)
    acc = accuracy_score(all_labels, all_preds)

    results_list.append({
        "Td": T_d,
        "Tw": T_w,
        "T_len": T_d - T_w + 1,
        "Model": model_type,
        "Accuracy": round(acc, 4),
        "Precision_macro": round(report["macro avg"]["precision"], 4),
        "Recall_macro": round(report["macro avg"]["recall"], 4),
        "F1_macro": round(report["macro avg"]["f1-score"], 4),
        "F1_weighted": round(report["weighted avg"]["f1-score"], 4),
    })

# === MAIN EVALUATION LOOP ===
for T_d in detection_times:
    for T_w in window_sizes:
        T_len = T_d - T_w + 1
        expected_shape = (T_len, NUM_FEATURES)
        folder_name = f"X_csv_split_{T_len}"
        test_path = os.path.join(base_dir, f"New_{T_d}", f"{T_w}", "split_tws", folder_name, "test")

        # === Load Test Data ===
        X_test_raw, y_test_raw = load_split_from_folder(test_path, expected_shape)
        label_encoder = LabelEncoder()
        label_encoder.fit(y_test_raw)

        test_dataset = MultiStreamDataset(X_test_raw, y_test_raw, label_encoder, augment=False)
        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # === Evaluate Teacher ===
        teacher_path = os.path.join(base_teacher_dir, f"mlpmixer_Td{T_d}_Tw{T_w}.pth")
        teacher = MLPMixerRansomwareClassifier(
            seq_len=T_len,
            feature_dim=NUM_FEATURES,
            num_classes=len(label_encoder.classes_),
            hidden_dim=128
        )
        evaluate_and_log(teacher, teacher_path, test_loader, label_encoder, T_d, T_w, "Teacher", results)

        # === Evaluate KD Student ===
        student_kd_path = os.path.join(student_kd_dir, f"student_from_mlp_Td{T_d}_Tw{T_w}.pth")
        student_kd = StudentCNN(input_length=T_len, num_classes=len(label_encoder.classes_)).to(DEVICE)
        evaluate_and_log(student_kd, student_kd_path, test_loader, label_encoder, T_d, T_w, "Student_KD", results)

        # === Evaluate Baseline Student ===
        student_b_path = os.path.join(student_baseline_dir, f"student_baseline_Td{T_d}_Tw{T_w}.pth")
        student_b = StudentCNN(input_length=T_len, num_classes=len(label_encoder.classes_)).to(DEVICE)
        evaluate_and_log(student_b, student_b_path, test_loader, label_encoder, T_d, T_w, "Student_Baseline", results)

# === Save to CSV ===
df_results = pd.DataFrame(results)
df_results.to_csv(csv_output_path, index=False)
print(f"\n✅ All evaluation results saved to: {csv_output_path}")


/tmp/ipykernel_2429355/2749317716.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DEVICE))
/tmp/ipykernel_2429


✅ All evaluation results saved to: /home/HardDisk/Satang/thesis_proj/Deep_Learning/cross_archi/mlp/results/model_eval_summary.csv


/tmp/ipykernel_2429355/2749317716.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=DEVICE))
